Importing required libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Getting data from wikipedia webpage and extracting table data

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#Creating BeautifulSoup object with lxml parser
soup = BeautifulSoup(source, 'lxml')

#Storing table data in a variable, first table appearance has the date
table = soup.find('table')

#Storing column header in a variable, from the header row of the wikipedia table
header = table.find_all('th')

columns = []

for head in header:
    columns.append(head.text.rstrip())
    
columns


['Postal code', 'Borough', 'Neighborhood']

#### Inserting table rows into a dataframe

In [33]:
#parsing through table rows to create a list and creating a dataframe
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.rstrip() for tr in td]
    l.append(row)
    
table_df = pd.DataFrame(l, columns = columns)
table_df.head()

,Postal code,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Rename column 'Postal code' to 'PostalCode'

In [34]:
table_df.rename(columns={'Postal code': 'PostalCode'}, inplace=True)
table_df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [35]:
#drop 'Not assigned' Boroughs
table_df = table_df[table_df['Borough'] != 'Not assigned']
table_df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights


In [37]:
#drop rows with 'None' values (first row has 'None' value for all columns)

table_df.dropna(inplace=True)
table_df.head()


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [38]:
#reset index
table_df.reset_index(drop=True, inplace=True)
table_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [15]:
#There is no PostalCode assigned to more than one Neighborhoods in the Wikipedia table data

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [16]:
#There is no Borough with 'Not assigned' Neighborhood

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [39]:
#Printing the no. of rows in the DataFrame
table_df.shape

(103, 3)

#### end of first section

## Adding latitude, longitude to the dataframe

##### importing lat, long csv file from http://cocl.us/Geospatial_data

In [23]:
loc_df = pd.read_csv('http://cocl.us/Geospatial_data')
loc_df.head()

#renaming column 'Postal Code' to 'PostalCode'
loc_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
loc_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Creating new dataframe with lat, long

In [40]:
neighborhood_df = pd.merge(left=table_df, right=loc_df)
neighborhood_df.shape
neighborhood_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


#### End of section 2